# Module 8: Histograms and Equalization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-graphics/blob/main/notebooks/08_Module.ipynb)

**Week 13: Histogram Calculation, Equalization, CLAHE, Histogram Matching**

## Learning Objectives
- Understand histogram theory and statistical representation
- Calculate and visualize image histograms
- Apply histogram equalization for contrast enhancement
- Use adaptive histogram equalization (CLAHE)
- Perform histogram-based image analysis and matching

---
## ⚠️ IMPORTANT: Run All Cells in Order

**This notebook must be executed sequentially from top to bottom.**

- Click **Runtime → Run all** (or **Cell → Run All**)
- Do NOT skip cells or run them out of order
- Each cell depends on variables from previous cells

---

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
print(f"OpenCV version: {cv2.__version__}")

---
## Mathematical Foundations: Histogram Theory

### What is a Histogram?

A **histogram** is a statistical representation of the distribution of pixel intensities.

**Formal definition**: For a grayscale image with intensity levels $[0, L-1]$:
$$
h(i) = \text{number of pixels with intensity } i
$$

For 8-bit images: $L = 256$, so $i \in [0, 255]$

**Normalized histogram** (probability distribution):
$$
p(i) = \frac{h(i)}{N}
$$

where $N$ is the total number of pixels:
$$
N = \sum_{i=0}^{L-1} h(i)
$$

### Why Histograms?

1. **Image analysis**: Understand intensity distribution
2. **Contrast assessment**: Identify low/high contrast images
3. **Thresholding**: Find optimal threshold values
4. **Enhancement**: Improve contrast via equalization
5. **Comparison**: Match or compare images
6. **Segmentation**: Separate objects by intensity

### Histogram Properties

**Mean intensity**:
$$
\mu = \sum_{i=0}^{L-1} i \cdot p(i)
$$

**Variance** (spread):
$$
\sigma^2 = \sum_{i=0}^{L-1} (i - \mu)^2 \cdot p(i)
$$

**Standard deviation**:
$$
\sigma = \sqrt{\sigma^2}
$$

**Entropy** (information content):
$$
H = -\sum_{i=0}^{L-1} p(i) \log_2 p(i)
$$

In [ ]:
# Load test images with different characteristics
import urllib.request
from urllib.request import Request, urlopen

# For dark image, we'll use the Cat image and make it darker
url_base = 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/320px-Cat03.jpg'
req_base = Request(url_base, headers={'User-Agent': 'Mozilla/5.0'})
with urlopen(req_base) as response:
    image_data = response.read()
with open('base.jpg', 'wb') as f:
    f.write(image_data)

# Load base image and create variations
img_base = cv2.imread('base.jpg', cv2.IMREAD_GRAYSCALE)

# Create dark image (darken the base image)
img_dark = (img_base * 0.4).astype(np.uint8)

# Bright image (brighten the base image)
img_bright = np.clip(img_base.astype(float) * 1.3 + 30, 0, 255).astype(np.uint8)

# Low contrast image (reduce contrast)
img_low_contrast = (img_base * 0.5 + 100).astype(np.uint8)

# Normal image
img_normal = img_base

print(f"Loaded images:")
print(f"  Dark image: {img_dark.shape}")
print(f"  Bright image: {img_bright.shape}")
print(f"  Low contrast image: {img_low_contrast.shape}")
print(f"  Normal image: {img_normal.shape}")

# Display
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

images = [img_dark, img_bright, img_low_contrast, img_normal]
titles = ['Dark Image', 'Bright Image', 'Low Contrast Image', 'Normal Image']

for i, (img, title) in enumerate(zip(images, titles)):
    axes[i].imshow(img, cmap='gray', vmin=0, vmax=255)
    axes[i].set_title(title)
    axes[i].axis('off')

plt.tight_layout()
plt.show()


### 1. Calculating Histograms

#### Manual Calculation

```python
histogram = np.zeros(256)
for pixel in image.flat:
    histogram[pixel] += 1
```

#### Using NumPy

```python
histogram, bins = np.histogram(image, bins=256, range=(0, 256))
```

#### Using OpenCV

```python
histogram = cv2.calcHist([image], [0], None, [256], [0, 256])
```

**Parameters**:
- `images`: List of source images
- `channels`: Channel indices (0 for grayscale)
- `mask`: Optional mask (None for whole image)
- `histSize`: Number of bins (256 for 8-bit)
- `ranges`: Intensity range ([0, 256] for 8-bit)

#### Color Histograms

For RGB images, compute 3 separate histograms:
$$
h_R(i), \quad h_G(i), \quad h_B(i)
$$

Or multi-dimensional histograms (2D, 3D).

In [ ]:
# Calculate and visualize histograms
print("=" * 70)
print("HISTOGRAM CALCULATION AND VISUALIZATION")
print("=" * 70)

def plot_histogram(img, title, ax):
    """Plot image and its histogram"""
    # Calculate histogram
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    
    # Calculate statistics
    mean_val = np.mean(img)
    std_val = np.std(img)
    min_val = np.min(img)
    max_val = np.max(img)
    
    # Plot histogram
    ax.plot(hist, color='black', linewidth=1.5)
    ax.fill_between(range(256), hist.flatten(), alpha=0.3, color='blue')
    ax.set_xlim([0, 256])
    ax.set_xlabel('Intensity')
    ax.set_ylabel('Frequency')
    ax.set_title(f'{title}\nμ={mean_val:.1f}, σ={std_val:.1f}, range=[{min_val},{max_val}]')
    ax.grid(True, alpha=0.3)
    
    # Mark mean
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean={mean_val:.1f}')
    ax.legend()
    
    return hist, mean_val, std_val

# Visualize
fig = plt.figure(figsize=(16, 12))

images_data = [
    (img_dark, 'Dark Image'),
    (img_bright, 'Bright Image'),
    (img_low_contrast, 'Low Contrast'),
    (img_normal, 'Normal Image')
]

print(f"\nImage statistics:\n")

for i, (img, title) in enumerate(images_data):
    # Show image
    ax_img = plt.subplot(4, 2, 2*i + 1)
    ax_img.imshow(img, cmap='gray', vmin=0, vmax=255)
    ax_img.set_title(title)
    ax_img.axis('off')
    
    # Show histogram
    ax_hist = plt.subplot(4, 2, 2*i + 2)
    hist, mean_val, std_val = plot_histogram(img, title, ax_hist)
    
    print(f"{title}:")
    print(f"  Mean: {mean_val:.2f}")
    print(f"  Std Dev: {std_val:.2f}")
    print(f"  Min: {img.min()}, Max: {img.max()}")
    print(f"  Dynamic Range: {img.max() - img.min()}")
    print()

plt.tight_layout()
plt.show()

print("Key Insight: Histogram shape reveals image characteristics!")

In [ ]:
# Color histogram
print("=" * 70)
print("COLOR HISTOGRAM (RGB)")
print("=" * 70)

# Load color image
img_color = cv2.imread('normal.jpg')
img_color_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)

# Calculate histograms for each channel
colors = ('r', 'g', 'b')
channel_names = ('Red', 'Green', 'Blue')

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Show original image
axes[0, 0].imshow(img_color_rgb)
axes[0, 0].set_title('Original Color Image')
axes[0, 0].axis('off')

# Plot RGB histograms together
for i, (color, name) in enumerate(zip(colors, channel_names)):
    hist = cv2.calcHist([img_color], [i], None, [256], [0, 256])
    axes[0, 1].plot(hist, color=color, linewidth=2, alpha=0.7, label=name)
    axes[0, 1].set_xlim([0, 256])
    axes[0, 1].set_xlabel('Intensity')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('RGB Histogram (Combined)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

# Plot individual channel histograms
for i, (color, name) in enumerate(zip(colors, channel_names)):
    hist = cv2.calcHist([img_color], [i], None, [256], [0, 256])
    ax = axes[1, i] if i < 2 else axes[1, 1]
    if i == 2:
        ax.clear()
    
    row = 1
    col = i
    if i >= 2:
        row = 1
        col = 1
        axes[1, 1].clear()
    
    axes[row, col].plot(hist, color=color, linewidth=2)
    axes[row, col].fill_between(range(256), hist.flatten(), alpha=0.3, color=color)
    axes[row, col].set_xlim([0, 256])
    axes[row, col].set_xlabel('Intensity')
    axes[row, col].set_ylabel('Frequency')
    axes[row, col].set_title(f'{name} Channel')
    axes[row, col].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nColor statistics:")
for i, name in enumerate(channel_names):
    channel_data = img_color[:, :, i]
    print(f"  {name}: mean={np.mean(channel_data):.2f}, std={np.std(channel_data):.2f}")

print("\nKey Insight: Each RGB channel has its own histogram!")

### 2. Histogram Equalization

**Goal**: Enhance contrast by spreading out intensity distribution.

#### Mathematical Foundation

**Cumulative Distribution Function (CDF)**:
$$
c(i) = \sum_{j=0}^{i} p(j)
$$

**Transformation function**:
$$
s = T(r) = (L-1) \cdot c(r)
$$

Where:
- $r$: Input intensity level
- $s$: Output intensity level  
- $L$: Number of intensity levels (256 for 8-bit)

**In discrete form**:
$$
s_k = (L-1) \sum_{j=0}^{k} \frac{n_j}{N} = \frac{L-1}{N} \sum_{j=0}^{k} n_j
$$

Where:
- $n_j$: Number of pixels with intensity $j$
- $N$: Total number of pixels

#### Algorithm

1. Calculate histogram $h(i)$
2. Calculate normalized histogram $p(i) = h(i) / N$
3. Calculate CDF: $c(i) = \sum_{j=0}^{i} p(j)$
4. Transform: $\text{output}[i] = \text{round}((L-1) \cdot c[\text{input}[i]])$

#### Properties

**Result**: Approximately uniform histogram (flat distribution)

**Ideal output**:
$$
p_{\text{ideal}}(i) = \frac{1}{L} \quad \forall i
$$

**Advantages**:
- Automatic (no parameters)
- Enhances global contrast
- Simple and fast

**Disadvantages**:
- May over-enhance noise
- May change brightness
- Global method (ignores local variations)

In [ ]:
# Histogram equalization
print("=" * 70)
print("HISTOGRAM EQUALIZATION")
print("=" * 70)

# Apply histogram equalization
img_eq = cv2.equalizeHist(img_dark)

# Manual implementation for demonstration
def manual_histogram_equalization(img):
    """Manual implementation of histogram equalization"""
    # Calculate histogram
    hist, _ = np.histogram(img.flatten(), bins=256, range=[0, 256])
    
    # Calculate CDF
    cdf = hist.cumsum()
    
    # Normalize CDF to [0, 255]
    cdf_normalized = (cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())
    cdf_normalized = cdf_normalized.astype(np.uint8)
    
    # Apply transformation
    img_eq_manual = cdf_normalized[img]
    
    return img_eq_manual, hist, cdf, cdf_normalized

img_eq_manual, hist_orig, cdf_orig, cdf_norm = manual_histogram_equalization(img_dark)

# Verify both methods give same result
print(f"\nOpenCV and manual implementation match: {np.allclose(img_eq, img_eq_manual)}")

# Visualize
fig = plt.figure(figsize=(16, 12))

# Original image
ax1 = plt.subplot(3, 3, 1)
ax1.imshow(img_dark, cmap='gray', vmin=0, vmax=255)
ax1.set_title('Original Image (Dark)')
ax1.axis('off')

# Original histogram
ax2 = plt.subplot(3, 3, 2)
ax2.bar(range(256), hist_orig, color='blue', alpha=0.7)
ax2.set_title(f'Original Histogram\nMean={np.mean(img_dark):.1f}')
ax2.set_xlabel('Intensity')
ax2.set_ylabel('Frequency')
ax2.set_xlim([0, 256])
ax2.grid(True, alpha=0.3)

# CDF
ax3 = plt.subplot(3, 3, 3)
ax3.plot(cdf_orig, color='blue', linewidth=2, label='CDF')
ax3.plot(cdf_norm, color='red', linewidth=2, linestyle='--', label='Normalized CDF')
ax3.set_title('Cumulative Distribution Function')
ax3.set_xlabel('Intensity')
ax3.set_ylabel('Cumulative Count')
ax3.set_xlim([0, 256])
ax3.legend()
ax3.grid(True, alpha=0.3)

# Equalized image
ax4 = plt.subplot(3, 3, 4)
ax4.imshow(img_eq, cmap='gray', vmin=0, vmax=255)
ax4.set_title('Equalized Image')
ax4.axis('off')

# Equalized histogram
ax5 = plt.subplot(3, 3, 5)
hist_eq = cv2.calcHist([img_eq], [0], None, [256], [0, 256])
ax5.bar(range(256), hist_eq.flatten(), color='green', alpha=0.7)
ax5.set_title(f'Equalized Histogram\nMean={np.mean(img_eq):.1f}')
ax5.set_xlabel('Intensity')
ax5.set_ylabel('Frequency')
ax5.set_xlim([0, 256])
ax5.grid(True, alpha=0.3)

# Comparison
ax6 = plt.subplot(3, 3, 6)
ax6.plot(hist_orig, color='blue', label='Original', alpha=0.7, linewidth=2)
ax6.plot(hist_eq, color='green', label='Equalized', alpha=0.7, linewidth=2)
ax6.set_title('Histogram Comparison')
ax6.set_xlabel('Intensity')
ax6.set_ylabel('Frequency')
ax6.set_xlim([0, 256])
ax6.legend()
ax6.grid(True, alpha=0.3)

# Side by side comparison
ax7 = plt.subplot(3, 3, 7)
comparison = np.hstack([img_dark, img_eq])
ax7.imshow(comparison, cmap='gray')
ax7.set_title('Before (Left) vs After (Right)')
ax7.axis('off')

# Transformation function visualization
ax8 = plt.subplot(3, 3, 8)
ax8.plot(cdf_norm, color='purple', linewidth=2)
ax8.plot([0, 255], [0, 255], 'k--', alpha=0.3, label='Identity')
ax8.set_title('Transformation Function T(r)')
ax8.set_xlabel('Input Intensity (r)')
ax8.set_ylabel('Output Intensity (s)')
ax8.set_xlim([0, 255])
ax8.set_ylim([0, 255])
ax8.legend()
ax8.grid(True, alpha=0.3)
ax8.set_aspect('equal')

plt.tight_layout()
plt.show()

print("\nContrast improvement:")
print(f"  Original range: [{img_dark.min()}, {img_dark.max()}] = {img_dark.max() - img_dark.min()}")
print(f"  Equalized range: [{img_eq.min()}, {img_eq.max()}] = {img_eq.max() - img_eq.min()}")
print(f"  Original std: {np.std(img_dark):.2f}")
print(f"  Equalized std: {np.std(img_eq):.2f}")

print("\nKey Insight: Equalization spreads out intensities for better contrast!")

### 3. Adaptive Histogram Equalization (CLAHE)

**CLAHE** = Contrast Limited Adaptive Histogram Equalization

#### Problems with Global Equalization

1. **Over-enhancement**: Amplifies noise
2. **Ignores local contrast**: Uses global statistics
3. **Artifacts**: Creates unnatural appearance

#### CLAHE Solution

**Adaptive**: Divide image into tiles, equalize each separately

**Contrast limiting**: Clip histogram to prevent over-amplification

#### Algorithm

1. **Divide** image into $M \times N$ tiles (e.g., 8×8)
2. For each tile:
   - Calculate histogram $h(i)$
   - **Clip** histogram at threshold:
     $$
     h_{\text{clipped}}(i) = \min(h(i), \text{clipLimit})
     $$
   - Redistribute excess pixels uniformly
   - Compute equalization
3. **Interpolate** between tiles (bilinear) to avoid boundary artifacts

#### Parameters

**clipLimit**: Contrast limiting threshold
- Higher → more contrast enhancement
- Lower → less enhancement, more natural
- Typical: 2.0 - 4.0

**tileGridSize**: Size of tiles
- Smaller tiles → more local adaptation
- Larger tiles → more global (closer to standard equalization)
- Typical: (8, 8)

#### CLAHE vs Standard Equalization

| Property | Standard | CLAHE |
|----------|----------|-------|
| Scope | Global | Local (tiles) |
| Noise | Amplifies | **Limits** |
| Contrast | High (may be excessive) | **Controlled** |
| Appearance | May be unnatural | **More natural** |
| Speed | Fast | Slower |
| Use case | Simple images | **Complex images** |

In [ ]:
# CLAHE (Contrast Limited Adaptive Histogram Equalization)
print("=" * 70)
print("CLAHE - ADAPTIVE HISTOGRAM EQUALIZATION")
print("=" * 70)

# Create CLAHE object
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
img_clahe = clahe.apply(img_dark)

# Compare different clipLimits
clip_limits = [1.0, 2.0, 4.0, 8.0]
clahe_results = []

print(f"\nTesting different clipLimit values:\n")

for clip_limit in clip_limits:
    clahe_temp = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    result = clahe_temp.apply(img_dark)
    clahe_results.append(result)
    print(f"clipLimit={clip_limit}: std={np.std(result):.2f}")

# Visualize comparison
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# Original
axes[0, 0].imshow(img_dark, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original (Dark)')
axes[0, 0].axis('off')

# Standard equalization
axes[0, 1].imshow(img_eq, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Standard Equalization')
axes[0, 1].axis('off')

# CLAHE (default)
axes[0, 2].imshow(img_clahe, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title('CLAHE (clipLimit=2.0)')
axes[0, 2].axis('off')

# Histograms
hist_dark = cv2.calcHist([img_dark], [0], None, [256], [0, 256])
hist_eq = cv2.calcHist([img_eq], [0], None, [256], [0, 256])
hist_clahe = cv2.calcHist([img_clahe], [0], None, [256], [0, 256])

axes[1, 0].plot(hist_dark, color='blue', linewidth=2)
axes[1, 0].set_title('Original Histogram')
axes[1, 0].set_xlim([0, 256])
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(hist_eq, color='green', linewidth=2)
axes[1, 1].set_title('Standard Equalization Histogram')
axes[1, 1].set_xlim([0, 256])
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].plot(hist_clahe, color='red', linewidth=2)
axes[1, 2].set_title('CLAHE Histogram')
axes[1, 2].set_xlim([0, 256])
axes[1, 2].grid(True, alpha=0.3)

# Different clipLimits
for i, (clip_limit, result) in enumerate(zip(clip_limits[:3], clahe_results[:3])):
    axes[2, i].imshow(result, cmap='gray', vmin=0, vmax=255)
    axes[2, i].set_title(f'CLAHE clipLimit={clip_limit}')
    axes[2, i].axis('off')

plt.tight_layout()
plt.show()

# Detail comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Crop a region for detail
h, w = img_dark.shape
crop = (slice(h//4, 3*h//4), slice(w//4, 3*w//4))

axes[0].imshow(img_dark[crop], cmap='gray')
axes[0].set_title('Original (Detail)')
axes[0].axis('off')

axes[1].imshow(img_eq[crop], cmap='gray')
axes[1].set_title('Standard Equalization (Detail)\nNote: May amplify noise')
axes[1].axis('off')

axes[2].imshow(img_clahe[crop], cmap='gray')
axes[2].set_title('CLAHE (Detail)\nNote: More natural appearance')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\nKey Insight: CLAHE provides better local contrast with less noise!")

### 4. Color Image Equalization

**Problem**: Cannot directly equalize RGB channels!

**Why?** Equalizing R, G, B independently changes color balance.

#### Solution 1: Convert to HSV

1. Convert RGB → HSV
2. Equalize **V (Value)** channel only
3. Convert back: HSV → RGB

**Preserves**: Hue and Saturation (color information)

**Enhances**: Brightness/Intensity

#### Solution 2: Convert to YCrCb

1. Convert RGB → YCrCb
2. Equalize **Y (Luminance)** channel only
3. Convert back: YCrCb → RGB

**YCrCb** separates:
- Y: Luminance (brightness)
- Cr, Cb: Chrominance (color)

#### Solution 3: Convert to LAB

1. Convert RGB → LAB
2. Equalize **L (Lightness)** channel only
3. Convert back: LAB → RGB

**LAB** is perceptually uniform:
- L: Lightness
- A: Green-Red
- B: Blue-Yellow

In [ ]:
# Color image equalization
print("=" * 70)
print("COLOR IMAGE EQUALIZATION")
print("=" * 70)

# Load color image (make it darker for demonstration)
img_color_dark = (img_color * 0.5).astype(np.uint8)
img_color_dark_rgb = cv2.cvtColor(img_color_dark, cv2.COLOR_BGR2RGB)

# Method 1: Wrong way (equalize each RGB channel)
img_rgb_eq = img_color_dark.copy()
for i in range(3):
    img_rgb_eq[:, :, i] = cv2.equalizeHist(img_color_dark[:, :, i])
img_rgb_eq_rgb = cv2.cvtColor(img_rgb_eq, cv2.COLOR_BGR2RGB)

# Method 2: HSV (equalize V channel)
img_hsv = cv2.cvtColor(img_color_dark, cv2.COLOR_BGR2HSV)
img_hsv[:, :, 2] = cv2.equalizeHist(img_hsv[:, :, 2])
img_hsv_eq = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
img_hsv_eq_rgb = cv2.cvtColor(img_hsv_eq, cv2.COLOR_BGR2RGB)

# Method 3: YCrCb (equalize Y channel)
img_ycrcb = cv2.cvtColor(img_color_dark, cv2.COLOR_BGR2YCrCb)
img_ycrcb[:, :, 0] = cv2.equalizeHist(img_ycrcb[:, :, 0])
img_ycrcb_eq = cv2.cvtColor(img_ycrcb, cv2.COLOR_YCrCb2BGR)
img_ycrcb_eq_rgb = cv2.cvtColor(img_ycrcb_eq, cv2.COLOR_BGR2RGB)

# Method 4: LAB (equalize L channel)
img_lab = cv2.cvtColor(img_color_dark, cv2.COLOR_BGR2LAB)
img_lab[:, :, 0] = cv2.equalizeHist(img_lab[:, :, 0])
img_lab_eq = cv2.cvtColor(img_lab, cv2.COLOR_LAB2BGR)
img_lab_eq_rgb = cv2.cvtColor(img_lab_eq, cv2.COLOR_BGR2RGB)

# Method 5: CLAHE on LAB
img_lab_clahe = cv2.cvtColor(img_color_dark, cv2.COLOR_BGR2LAB)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
img_lab_clahe[:, :, 0] = clahe.apply(img_lab_clahe[:, :, 0])
img_lab_clahe_result = cv2.cvtColor(img_lab_clahe, cv2.COLOR_LAB2BGR)
img_lab_clahe_rgb = cv2.cvtColor(img_lab_clahe_result, cv2.COLOR_BGR2RGB)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(16, 11))

axes[0, 0].imshow(img_color_dark_rgb)
axes[0, 0].set_title('Original (Dark)')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_rgb_eq_rgb)
axes[0, 1].set_title('RGB Channels Equalized\n⚠️ WRONG: Color shift!')
axes[0, 1].axis('off')

axes[0, 2].imshow(img_hsv_eq_rgb)
axes[0, 2].set_title('HSV (V channel)\n✓ Preserves color')
axes[0, 2].axis('off')

axes[1, 0].imshow(img_ycrcb_eq_rgb)
axes[1, 0].set_title('YCrCb (Y channel)\n✓ Good separation')
axes[1, 0].axis('off')

axes[1, 1].imshow(img_lab_eq_rgb)
axes[1, 1].set_title('LAB (L channel)\n✓ Perceptually uniform')
axes[1, 1].axis('off')

axes[1, 2].imshow(img_lab_clahe_rgb)
axes[1, 2].set_title('LAB + CLAHE\n✓✓ BEST: Natural enhancement')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nRecommendations:")
print("  ❌ Never equalize RGB channels independently")
print("  ✓ HSV: Simple, preserves hue")
print("  ✓ YCrCb: Good for video")
print("  ✓ LAB: Perceptually uniform")
print("  ✓✓ LAB + CLAHE: Best for most applications")

print("\nKey Insight: Use color spaces that separate luminance from chrominance!")

## Summary

### Key Concepts

1. **Histogram**: Statistical representation of intensity distribution
2. **Histogram Equalization**: Enhance contrast by spreading intensities
3. **CLAHE**: Adaptive equalization with contrast limiting
4. **Color Equalization**: Separate luminance from chrominance

### Mathematical Formulas Reference

**Histogram**:
$$
h(i) = \text{count of pixels with intensity } i
$$

**Normalized histogram**:
$$
p(i) = \frac{h(i)}{N}
$$

**CDF**:
$$
c(i) = \sum_{j=0}^{i} p(j)
$$

**Equalization transformation**:
$$
s = T(r) = (L-1) \cdot c(r)
$$

**Mean**:
$$
\mu = \sum_{i=0}^{L-1} i \cdot p(i)
$$

**Variance**:
$$
\sigma^2 = \sum_{i=0}^{L-1} (i-\mu)^2 \cdot p(i)
$$

### Method Comparison

| Method | Scope | Noise | Best For |
|--------|-------|-------|----------|
| **Standard Equalization** | Global | Amplifies | Simple, low-noise images |
| **CLAHE** | Local | **Limits** | Complex images, medical imaging |
| **HSV Equalization** | V channel | Varies | Color photos (preserves hue) |
| **YCrCb Equalization** | Y channel | Varies | Video processing |
| **LAB + CLAHE** | L channel | **Limits** | **Best overall** |

### OpenCV Functions Reference

| Operation | Function | Key Parameters |
|-----------|----------|----------------|
| Calculate histogram | `cv2.calcHist([img], [0], None, [256], [0,256])` | channels, bins, range |
| Equalize | `cv2.equalizeHist(img)` | Grayscale only |
| Create CLAHE | `cv2.createCLAHE(clipLimit, tileGridSize)` | clipLimit ≈ 2.0 |
| Apply CLAHE | `clahe.apply(img)` | Returns equalized image |

### Best Practices

1. **Grayscale**: Use standard equalization or CLAHE directly
2. **Color**: Convert to HSV/YCrCb/LAB, equalize intensity channel only
3. **Noisy images**: Use CLAHE (lower clipLimit)
4. **Medical images**: CLAHE with small tiles
5. **Natural photos**: LAB + CLAHE (clipLimit ≈ 2.0-3.0)

**Next**: Module 9 - Image Segmentation